In [6]:
from pyhdf.SD  import *
import numpy as np 
import matplotlib.pyplot as plt
import os
from skimage.measure import compare_ssim
import pandas as pd
print(os.getcwd())

def readfile_br(path,start,end):
    sequence=[]
    count=end-start
    for i in range(start,end):
        thispath=path+"br002_"+str(i)+".hdf"
        #print("Reading Files :",thispath)
        image_sequence = SD(thispath, SDC.READ)
        sds_obj = image_sequence.select('Data-Set-2')
        dim3 = sds_obj.get()
        frame=[]
        for i in range(0,141):
            frame.append(dim3[:,:,i])
        frame=np.array(frame)
        sequence.append(frame)
    sequence=np.array(sequence)
    data=sequence.reshape(count,141,128,110,1)
    #data=pad_the_frame(data)
    return data

def scale(data):
    from sklearn.preprocessing import minmax_scale
    shape = data.shape
    data = minmax_scale(data.ravel(), feature_range=(0,255)).reshape(shape)
    return data


def draw_ssim_graph(data):
    ssim=[]
    for i in range(0,140):
        ssim.append(compare_ssim(data[0,1,:,:,0],data[0,i + 1,:,:,0],multichannel=False))
    return ssim
def generatecsv(sample_num):
    start=1
    end=sample_num+1
    sequence_length=10
    path="../data/sampleData/"
    frame_start=0
    frame_end=141
    data=scale(readfile_br(path,start,end))
    pd.DataFrame(draw_ssim_graph(data),columns=['SSIM'] ).to_csv("ssim_graph.csv",index=False)



/Users/agupta/Documents/DM LAB/projects/e3d_lstm-master/Presentable Experiments


In [7]:
generatecsv(1)

/Users/agupta/Documents/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.


In [8]:

chart_data=pd.read_csv("ssim_graph.csv")

In [9]:
chart_data

,SSIM
0,1.000000
1,0.989183
2,0.960779
3,0.921473
4,0.876256
...,...
135,0.027438
136,0.027368
137,0.027293
138,0.027215


In [23]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=chart_data.index, y=chart_data['SSIM'], fill='tozerox',text="Solar Radii , SSIM",hoverinfo='text+x+y')) # fill down to xaxis
fig.update_xaxes(title_text='Time',showgrid=False)
fig.update_yaxes(title_text='Value A',showgrid=False)
fig.update_layout(
    title = "Baseline Structured Similarity Index Measure(SSIM)",template="plotly_dark"
    
)

fig.show()

In [12]:
import plotly.express as px


fig = px.area(chart_data, y="SSIM", 
                title="Baseline Structured Similarity Index Measure(SSIM)",template="plotly_dark",showgrid=False)
fig.show()

TypeError: area() got an unexpected keyword argument 'showgrid'